# TensorFlow v2.15.0 - Apple Silicon

**Apple Silicon users:** Install TensorFlow using `pip install tensorflow-macos`.

References:
1. [Tensorflow 2.15 Release Notes](https://github.com/tensorflow/tensorflow/releases)
2. [Get started with tensorflow-metal](https://developer.apple.com/metal/tensorflow-plugin/)

Baseline CPU vs GPU:
1. Kaggle: [Tensorflow 2.15 CPU/GPU Comparison](https://www.kaggle.com/code/gdataranger/tensorflow-2-16-cpu-gpu-comparison)
2. Colab: [Tensorflow 2.15 CPU/GPU Comparison](https://colab.research.google.com/drive/1FyQ1VqYjsGCGtUcNwM0FkGRPFnTIQrR1)

## Setup

Start from a fresh virtual environment (venv) in order to ensure we have the correct requirements
installed for `v2.15.0`, as the differences from `v2.15.0` to `v2.15.1` have vastly different requirements.
<br>

```zsh
~ % cd ~/.virtualenvs
.virtualenvs % python3 -m venv venv-tf15
.virtualenvs % source venv-tf15/bin/activate
(venv-tf15) .virtualenvs % pip install --upgrade pip
(venv-tf15) .virtualenvs % pip list

Package    Version
---------- -------
pip        24.0
setuptools 65.5.0

(venv-tf15) ~ % pip install tensorflow-macos==2.15.0
(venv-tf15) ~ % pip install tensorflow-metal==1.1.0
(venv-tf15) ~ % pip install tensorflow==2.15.0
(venv-tf15) ~ % pip freeze > requirements_tf215.txt
(venv-tf15) ~ % pip list | grep -E 'keras|tensorflow'
keras                        2.15.0
tensorflow                   2.15.0
tensorflow-estimator         2.15.0
tensorflow-io-gcs-filesystem 0.36.0
tensorflow-macos             2.15.0
tensorflow-metal             1.1.0

(venv-tf15) ~ % jupyter notebook --port=8890
```

Now let's ensure we have this virtual environment available as a Jupyter kernel.

```zsh
(venv-tf15) .virtualenvs % pip install ipykernel
(venv-tf15) .virtualenvs % python3 -m ipykernel install --user --name=venv-tf15 --display-name="Python (TensorFlow 2.15)"
Installed kernelspec venv-tf15 in /Users/marksusol/Library/Jupyter/kernels/venv-tf15

(venv-tf15) .virtualenvs % jupyter kernelspec list
Available kernels:
ir           /Users/marksusol/Library/Jupyter/kernels/ir
venv-tf15    /Users/marksusol/Library/Jupyter/kernels/venv-tf15
python3      /Library/Frameworks/Python.framework/Versions/3.11/share/jupyter/kernels/python3
```

## JetBrain DataSpell

Ensure that **DataSpell** is correctly configured to use the `venv-tf15` virtual environment and properly loads the python/tensorflow versions.

![tensorflow-dataspell-tf15](../images/tensorflow-dataspell-tf15.png)

## Tensorflow

In [1]:
import sys
import tensorflow as tf

# Ensure the notebook runs on the correct environment in DataSpell.
assert sys.executable == '/Users/marksusol/.virtualenvs/venv-tf15/bin/python3'
assert tf.__version__ == '2.15.0'

print(f'System: {sys.executable}')
print(f'TensorFlow: {tf.__version__}')

System: /Users/marksusol/.virtualenvs/venv-tf15/bin/python3
TensorFlow: 2.15.0


### GPU Test

In [4]:
%%time

import tensorflow as tf

import os
# When = 3, the messages (1 - informational(I), 2 - warnings(W) and 3- errors(E)) will not be logged during code execution.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Ensure we see the GPU in device list.
print('Visible Devices: ', tf.config.get_visible_devices())

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=128)

Visible Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
391/391 [==============================] - 42s 96ms/step - loss: 4.5789 - accuracy: 0.0778
CPU times: user 28.8 s, sys: 8.66 s, total: 37.5 s
Wall time: 43.1 s


### CPU Testing

This test can be run by itself after restarting the runtime to ensure we don't run into the `cannot modify virtual devices once initialized.` message. We're not able to 'renable' the GPU device once disabled.

**Note:** You may have your jupyter notebook running on a different port, for this notebook we are running the notebook on `--port=8890` by default in **DataSpell**.

In [5]:
%%capture
!jupyter notebook stop 8890

In [2]:
%%time

import tensorflow as tf
# Removes GPU from list, i.e. []
tf.config.set_visible_devices([], 'GPU')
print('Visible Devices: ', tf.config.get_visible_devices())

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=128)

Visible Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
391/391 [==============================] - 393s 1s/step - loss: 4.6609 - accuracy: 0.0720
CPU times: user 17min 44s, sys: 2min 35s, total: 20min 19s
Wall time: 6min 34s
